In [2]:
import numpy as np 
import pandas as pd 
import nltk

In [3]:
hate_train = pd.read_csv('hate_speech.csv')
hate_test = pd.read_csv('hate_speech_test.csv')

In [5]:
hate_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [6]:
hate_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


In [7]:
import re

In [8]:
def clean_text(df,col):
    df[col] = df[col].str.lower()
    df[col] = df[col].apply(lambda e: re.sub(r" (@[A-Za-z0-9]+)  |  ([^0-9A-Za-z \t])  |  (\w+:\/\/\S+)  |  ^rt  |  http.+?","",e))
    return df     

hate_train_final = clean_text(hate_train,'tweet')
hate_test_final =  clean_text(hate_test,'tweet')

In [10]:
hate_train_final

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [11]:
from sklearn.utils import resample

In [14]:
train_majority = hate_train_final[hate_train_final['label'] ==  0]
train_minority = hate_train_final[hate_train_final['label'] ==  1]

In [17]:
train_minority_upsampled = resample(train_minority,
                                   replace = True,
                                   n_samples = len(train_majority)
                                   )

train_upsampled = pd.concat([train_minority_upsampled,train_majority])

train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier

In [19]:
pipeline = Pipeline([
        ('cv',CountVectorizer()),
        ('tfidf',TfidfTransformer()),
        ('sgd',SGDClassifier())
])

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],train_upsampled['label'],random_state = 0)


In [21]:
model = pipeline.fit(X_train,y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9680900672839918